In [2]:
import duckdb

con = duckdb.connect(
    "/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/warehouse/plt.duckdb"
)

con.execute("""
PRAGMA table_info('analytics_staging.stg_transfer_data');
""").df()

,cid,name,type,notnull,dflt_value,pk
0,0,club_name,VARCHAR,False,None,False
1,1,player_name,VARCHAR,False,None,False
2,2,age,VARCHAR,False,None,False
3,3,position,VARCHAR,False,None,False
4,4,club_involved_name,VARCHAR,False,None,False
5,5,fee,VARCHAR,False,None,False
6,6,transfer_type,VARCHAR,False,None,False
7,7,transfer_movement,VARCHAR,False,None,False
8,8,transfer_period,VARCHAR,False,None,False
9,9,fee_cleaned,VARCHAR,False,None,False


In [3]:
from pathlib import Path
import duckdb

# 1) Connect to the SAME DuckDB file your dbt profile uses
con = duckdb.connect(
    "/Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/warehouse/plt.duckdb"
)  # adjust if needed

# 2) Source relation (from your table list)
SRC = "analytics_staging.stg_transfer_data"

# 3) Output directory (relative to this notebook's folder)
OUT_DIR = Path.cwd().parent / "dbt" / "seeds" / "split_files"
OUT_DIR.mkdir(parents=True, exist_ok=True)
print("Writing to:", OUT_DIR)

# 4) Discover available years in the source
years = [
    r[0]
    for r in con.execute(f"""
    with y as (
      select
        coalesce(
          extract(year from try_cast(year as date)),
          try_cast(regexp_extract(season, '^[0-9]{{4}}') as int)
        ) as yr
      from {SRC}
    )
    select distinct yr from y where yr is not null order by 1;
""").fetchall()
]
print("Years found:", years)

# 5) Export one CSV per year
for y in years:
    out_path = (OUT_DIR / f"premier_league_{y}.csv").as_posix()
    con.execute(f"""
      copy (
        select *
        from {SRC}
        where coalesce(
          extract(year from try_cast(year as date)),
          try_cast(regexp_extract(season, '^[0-9]{{4}}') as int)
        ) = {y}
      )
      to '{out_path}' (header, delimiter ',');
    """)
    print("Wrote:", out_path)

Writing to: /Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/dbt/seeds/split_files
Years found: [1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
Wrote: /Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/dbt/seeds/split_files/premier_league_1992.csv
Wrote: /Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/dbt/seeds/split_files/premier_league_1993.csv
Wrote: /Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/dbt/seeds/split_files/premier_league_1994.csv
Wrote: /Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/dbt/seeds/split_files/premier_league_1995.csv
Wrote: /Users/mohammed/repos/GreatLockIn2025/dbt_projects/dbt_football_project/DBT_PFL_Statistics/d